<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Javiera Palominos Clarke
- Nombre de alumno 2: Juan Carlos Araya


### **Link de repositorio de GitHub:** `https://github.com/kuanki1/MDS7202Labs/blob/main/Lab7/Laboratorio7_enunciado.ipynb`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
### Código aquí ###
labels = np.where(df_players['National_Position'].isna(), 0, 1)
labels = pd.Series(labels) 

In [5]:
labels.groupby(labels).count()

0    16513
1     1075
dtype: int64

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [6]:
### Código aquí ###
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Name', 'Nationality','Club_Position','Preffered_Foot','Work_Rate']),
        ('StandardScaler', StandardScaler(),['Height', 'Weight', 'Age']), 
        ('MinMaxScaler',MinMaxScaler(),['Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
        )])

Variables categoricas deben ser codificadas, variables descriptivas tipo edad, peso y altura que distribuyen normalmente, se centran en 0 y se ajusta para SD = 1. El resto de cualidades en cuanto a habilidad que van entre 0 y 100 de ajustan entre 0 y 1 (max-min) pues no necesariamente se distribuyen de manera normal sobre los datos.

In [7]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
pipeline_1 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', LinearSVC())
])
pipeline_2 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', KNeighborsClassifier())
])
pipeline_3 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', RandomForestClassifier())
])

Siguiendo el chart, tenemos la cantidad de data suficiente (y etiquetada) para una clasificacion categorica y binaria, por lo tanto el primer modelo sera un SVC. Para los siguientes dos se eligio un modelo de KNN, y finalmente un RandomForest.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [8]:
### Código aquí ###
X = df_players.drop(columns=['National_Position'])
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)

In [17]:
X_train

,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
15906,Katsuya Nagato,Japan,LM,172,71,Left,22,Medium / Medium,3,2,...,54,64,36,21,24,20,27,33,30,22
4272,André Hoffmann,Germany,Sub,188,84,Right,24,Medium / Medium,2,2,...,58,59,72,61,29,34,27,31,42,23
2440,Craig Gardner,England,RS,176,81,Right,30,High / Medium,3,2,...,69,66,68,87,65,84,70,76,81,71
5201,Nicolás Colazo,Argentina,Sub,175,72,Left,26,Medium / Low,3,3,...,77,70,62,76,54,70,77,71,69,46
2081,Arturo Mina,Ecuador,Sub,189,89,Right,26,High / Medium,3,2,...,40,67,79,52,30,33,27,41,45,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590,Peter Torreiro,Brazil,GK,185,84,Right,25,Medium / Medium,1,1,...,45,67,20,14,11,18,16,12,25,8
6777,Ramiro Carrera,Argentina,LM,182,72,Right,25,Medium / Medium,4,3,...,70,91,64,71,64,66,72,73,65,56
16199,Matt Acton,Australia,Sub,184,85,Right,24,Medium / Medium,2,1,...,38,59,19,16,19,18,20,18,27,11
4129,Mario Maloča,Croatia,CB,190,83,Right,27,Medium / High,2,2,...,61,76,72,40,40,27,13,36,52,23


In [9]:
pipeline_1.fit(X_train,y_train)
pipeline_2.fit(X_train,y_train)
pipeline_3.fit(X_train,y_train)

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Name', 'Nationality',
                                                   'Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', 'Volleys'])])),
                ('Modelo', RandomForestClassifier())])

In [10]:
y_pred_1 = pipeline_1.predict(X_test)
y_pred_2 = pipeline_2.predict(X_test)
y_pred_3 = pipeline_3.predict(X_test)

In [11]:
print('Classification Report for Pipeline 1')
print(classification_report(y_test, y_pred_1))

Classification Report for Pipeline 1
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5450
           1       0.72      0.34      0.46       355

    accuracy                           0.95      5805
   macro avg       0.84      0.66      0.72      5805
weighted avg       0.94      0.95      0.94      5805



In [12]:
print('Classification Report for Pipeline 2')
print(classification_report(y_test, y_pred_2))

Classification Report for Pipeline 2
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.54      0.08      0.13       355

    accuracy                           0.94      5805
   macro avg       0.74      0.54      0.55      5805
weighted avg       0.92      0.94      0.92      5805



In [13]:
print('Classification Report for Pipeline 3')
print(classification_report(y_test, y_pred_3))

Classification Report for Pipeline 3
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.81      0.04      0.07       355

    accuracy                           0.94      5805
   macro avg       0.88      0.52      0.52      5805
weighted avg       0.93      0.94      0.91      5805



**Respuesta**:

- Accuracy: Cantidad de predicciones correctas.
- Precision: es la fracción entre las instancias relevantes (positivos) predichas correctamente y el total de las predicciones relevantes realizadas.
- Recall: es la fracción entre las instancias relevantes (positivos) predichas y las que se deberían haber predicho.

Respecto a la mejora de los resultados, estos si pueden mejorarse, esto puede ser mediante un preprocesamiento, por ejemplo aplicando transformaciones para poder identificar de mejor manera la distribución de las características, eliminando redundancias o diferenciando valores aparentemente similares. Por otro lado existe un desbalance en las etiquetas del entrenamiento, lo que puede producir que el predictor se overfitee, por lo que sería relevante obtener más datos con la etiqueta 1.

Respecto a la cantidad de ejemplos por clase, cómo se mencionó anteiormente esta si influye, ya que esta puede provocar que el predictor se overfitee.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [14]:
### Código aquí ###
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

In [15]:
### Código aquí ###
df_filtered = df_players.query("Club_Position != 'Sub' and Club_Position != 'Res' ")
df_filtered

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17310,Tom Bolarinwa,England,NaN,RM,180,80,Right,26,Medium / Medium,3,...,54,54,22,38,37,37,37,32,29,29
17312,Zak Mills,England,NaN,RB,182,83,Right,24,Medium / Medium,4,...,58,61,43,22,24,28,30,32,39,31
17442,Wataru Hashimoto,Japan,NaN,LB,181,72,Left,30,Medium / High,2,...,64,71,41,29,26,26,25,25,31,29
17461,Callum Cooke,England,NaN,LCM,172,72,Right,20,Medium / Medium,3,...,60,63,57,48,50,54,42,33,69,40


In [102]:
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(ataque) , 'ataque',df_filtered['Club_Position'])
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(central_ataque) , 'central_ataque',df_filtered['label'])
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(central) , 'central',df_filtered['label'])
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(central_defensa) , 'central_defensa',df_filtered['label'])
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(defensa) , 'defensa',df_filtered['label'])
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'Club_Position'].isin(arquero) , 'arquero',df_filtered['label'])

In [103]:
valid = ['ataque','central_ataque','central','central_defensa','defensa','arquero']
df_filtered.loc[:,'label']= np.where(df_filtered.loc[:,'label'].isin(valid) , df_filtered['label'],np.nan)

In [104]:
#AÑADIDO POST PRIMER ENTRENAMIENTO, se modifica los nans para no ser eliminados.
df_filtered.loc[:,'National_Position']= np.where(df_filtered.loc[:,'National_Position'].isna() , 'Not Selected',df_filtered['National_Position'])

In [105]:
X = df_filtered.dropna()
X = X.drop(columns=['Club_Position'])
labels = X['label']
X = X.drop(columns=['label'])
labels.groupby(labels).count()

label
arquero             632
ataque              430
central             907
central_ataque      581
central_defensa     209
defensa            1180
Name: label, dtype: int64

In [106]:
X.columns

Index(['Name', 'Nationality', 'National_Position', 'Height', 'Weight',
       'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot', 'Skill_Moves',
       'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')

In [107]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Name', 'Nationality','National_Position','Preffered_Foot','Work_Rate']),
        ('StandardScaler', StandardScaler(),['Height', 'Weight', 'Age']), 
        ('MinMaxScaler',MinMaxScaler(),['Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
        )])

In [108]:
pipeline_4 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', LinearSVC())
])
pipeline_5 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', KNeighborsClassifier())
])
pipeline_6 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Modelo', RandomForestClassifier())
])

Al igual que en la situacion anterior, tenemos una cantidad de data suficiente (y etiquetada) para una clasificacion categorica, por lo tanto el primer modelo sera un SVC. Para los siguientes dos se eligio un modelo de KNN, y finalmente un RandomForest.

In [109]:
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)

In [110]:
pipeline_4.fit(X_train,y_train)
pipeline_5.fit(X_train,y_train)
pipeline_6.fit(X_train,y_train)

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Name', 'Nationality',
                                                   'National_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['Weak_foot', 'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', 'Volleys'])])),
                ('Modelo', RandomForestClassifier())])

In [111]:
y_pred_4 = pipeline_4.predict(X_test)
y_pred_5 = pipeline_5.predict(X_test)
y_pred_6 = pipeline_6.predict(X_test)

In [112]:
print('Classification Report for Pipeline 4')
print(classification_report(y_test, y_pred_4))

Classification Report for Pipeline 4
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       158
         ataque       0.75      0.84      0.79       108
        central       0.59      0.61      0.60       227
 central_ataque       0.48      0.30      0.37       145
central_defensa       0.45      0.17      0.25        52
        defensa       0.80      0.98      0.88       295

       accuracy                           0.74       985
      macro avg       0.68      0.65      0.65       985
   weighted avg       0.71      0.74      0.72       985



In [113]:
print('Classification Report for Pipeline 5')
print(classification_report(y_test, y_pred_5))

Classification Report for Pipeline 5
                 precision    recall  f1-score   support

        arquero       0.99      1.00      1.00       158
         ataque       0.51      0.60      0.55       108
        central       0.48      0.54      0.51       227
 central_ataque       0.41      0.22      0.29       145
central_defensa       0.23      0.13      0.17        52
        defensa       0.71      0.81      0.76       295

       accuracy                           0.63       985
      macro avg       0.55      0.55      0.55       985
   weighted avg       0.61      0.63      0.62       985



In [114]:
print('Classification Report for Pipeline 6')
print(classification_report(y_test, y_pred_6))

Classification Report for Pipeline 6
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       158
         ataque       0.76      0.82      0.79       108
        central       0.53      0.63      0.58       227
 central_ataque       0.38      0.16      0.22       145
central_defensa       0.60      0.06      0.11        52
        defensa       0.76      0.97      0.86       295

       accuracy                           0.71       985
      macro avg       0.67      0.61      0.59       985
   weighted avg       0.68      0.71      0.67       985



**Respuesta**:

Para mejorar los resultados, se modifico la columna de National_Position, dado que como contenía muchisimos NAN, disminuía muchisimo la cantidad de datos para entrenar. Los nans fueron reemplzados por un simple 'Not Selected' y fue simplemente entregado como otro parametro mas de clasificacion.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [115]:
### Código aquí ###
sueldos = pd.read_csv("sueldos.csv")
sueldos = sueldos.drop("Unnamed: 0", 1)
sueldos = sueldos.rename(columns={"Player": "Name"})
sueldos

,Name,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [116]:
dfs = pd.merge(left=df_players,
                    right=sueldos,
                    on="Name",
                    sort=True,
                    how="outer")
dfs = dfs.dropna(subset="Weekly Salary", axis=0)
dfs = dfs.dropna(subset="Height", axis=0)
dfs.describe()

,Height,Weight,Age,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Weekly Salary
count,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.00000,...,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1.866000e+03
mean,182.366559,76.266345,24.930868,3.052519,2.457663,65.370311,61.552519,48.619507,50.511790,53.00000,...,68.071811,56.762594,61.620043,49.897642,53.172026,53.268489,47.702036,52.474812,48.392283,5.363130e+04
std,6.546156,7.033456,4.131101,0.680976,0.856815,18.225297,20.614518,24.436182,24.733586,24.68378,...,11.157666,19.228412,18.777998,21.645839,20.897269,20.428841,19.502833,16.936046,20.205789,8.193256e+04
min,162.000000,48.000000,17.000000,1.000000,1.000000,10.000000,5.000000,5.000000,8.000000,9.00000,...,30.000000,8.000000,11.000000,5.000000,5.000000,6.000000,7.000000,11.000000,5.000000,6.200000e+02
25%,178.000000,72.000000,22.000000,3.000000,2.000000,62.000000,56.000000,23.000000,24.000000,28.00000,...,61.000000,49.000000,52.000000,32.000000,38.000000,40.000000,33.250000,42.000000,32.250000,1.413700e+04
50%,183.000000,76.000000,25.000000,3.000000,2.000000,71.000000,68.000000,56.000000,59.000000,63.00000,...,69.000000,61.000000,68.000000,54.000000,59.000000,58.000000,48.000000,54.000000,51.000000,2.940400e+04
75%,187.000000,81.000000,28.000000,3.000000,3.000000,77.000000,75.000000,71.000000,72.000000,74.00000,...,75.000000,70.000000,75.750000,68.000000,69.000000,69.000000,64.000000,65.000000,65.000000,6.502900e+04
max,201.000000,99.000000,39.000000,5.000000,5.000000,95.000000,97.000000,92.000000,91.000000,92.00000,...,95.000000,93.000000,92.000000,95.000000,91.000000,90.000000,91.000000,92.000000,91.000000,1.538905e+06


In [117]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Nationality','National_Position','Preffered_Foot','Work_Rate']),
        ('StandardScaler', StandardScaler(),['Height', 'Weight', 'Age']), 
        ('MinMaxScaler',MinMaxScaler(),['Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
        )])

In [118]:
df_final = dfs.copy()
X = df_final.drop(["Name", "Weekly Salary"], 1)
Y = dfs["Weekly Salary"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=24)

A partir del cheat-sheet recomendado se eligió el regresor ElasticNet.

In [119]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score

pipeline_7 =Pipeline([('ColumnTransform', preprocessing_transformer) , 
                       ('Regresor', ElasticNet(alpha=0.1))])

pipeline_7.fit(X_train, y_train)

y_pred_7 = pipeline_7.predict(X_test)

print(f" R² score: {r2_score(y_test, y_pred_7)}")   

 R² score: 0.36002372721293807


**Respuesta**:

R-cuadrado es una medida estadística de qué tan cerca están los datos de la línea de regresión ajustada, por lo que su escala posee como mejor puntaje el 1. Se pueden mejorar los resultados con una mayor cantidad de filas para realizar un mejor entrenamiento o con una creación de variables mediante estadísticos a partir de las variables disponibles.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>